In [1]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
loader = TextLoader("./demo2.txt")
docs = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=40,
    separators=["\n", "。", "！", "？", "，", "、", ""]
)

In [4]:
texts = text_splitter.split_documents(docs)

In [5]:
embeddings_model = OpenAIEmbeddings()

In [6]:
db = FAISS.from_documents(texts, embeddings_model)

In [7]:
retriever = db.as_retriever()

In [8]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [9]:
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history', output_key='answer')

In [10]:
qa = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    memory=memory
)

In [11]:
question = "卢浮宫这个名字怎么来的？"
qa.invoke({"chat_history": memory, "question": question})

{'chat_history': [HumanMessage(content='卢浮宫这个名字怎么来的？'),
  AIMessage(content='根据法国百科全书辞典大拉鲁斯百科全书（Grand Larousse encyclopédique）的说法，"罗浮宫"这个名字来源于与狼狩猎巢穴的联系（拉丁语：lupus，下帝国：lupara）。')],
 'question': '卢浮宫这个名字怎么来的？',
 'answer': '根据法国百科全书辞典大拉鲁斯百科全书（Grand Larousse encyclopédique）的说法，"罗浮宫"这个名字来源于与狼狩猎巢穴的联系（拉丁语：lupus，下帝国：lupara）。'}

In [12]:
question = "对应的拉丁语是什么呢？"
qa.invoke({"chat_history": memory, "question": question})

{'chat_history': [HumanMessage(content='卢浮宫这个名字怎么来的？'),
  AIMessage(content='根据法国百科全书辞典大拉鲁斯百科全书（Grand Larousse encyclopédique）的说法，"罗浮宫"这个名字来源于与狼狩猎巢穴的联系（拉丁语：lupus，下帝国：lupara）。'),
  HumanMessage(content='对应的拉丁语是什么呢？'),
  AIMessage(content='这个名字对应的拉丁语是"lupus"。')],
 'question': '对应的拉丁语是什么呢？',
 'answer': '这个名字对应的拉丁语是"lupus"。'}

In [13]:
qa = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    memory=memory,
    return_source_documents=True
)

In [14]:
question = "卢浮宫这个名字怎么来的？"
qa.invoke({"chat_history": memory, "question": question})

{'chat_history': [HumanMessage(content='卢浮宫这个名字怎么来的？'),
  AIMessage(content='根据法国百科全书辞典大拉鲁斯百科全书（Grand Larousse encyclopédique）的说法，"罗浮宫"这个名字来源于与狼狩猎巢穴的联系（拉丁语：lupus，下帝国：lupara）。'),
  HumanMessage(content='对应的拉丁语是什么呢？'),
  AIMessage(content='这个名字对应的拉丁语是"lupus"。'),
  HumanMessage(content='卢浮宫这个名字怎么来的？'),
  AIMessage(content='这个名字对应的拉丁语是lupus。')],
 'question': '卢浮宫这个名字怎么来的？',
 'answer': '这个名字对应的拉丁语是lupus。',
 'source_documents': [Document(page_content='“罗浮宫”这个名字的由来有些争议。根据法国百科全书辞典大拉鲁斯百科全书（Grand Larousse encyclopédique）的说法，这个名字来源于与狼狩猎巢穴的联系（拉丁语：lupus，下帝国： lupara）。[6]此后好几个世纪，罗浮宫发生了很大变化。14世纪，法王查理五世觉得罗浮宫堡比位于塞纳河当中之城岛（西岱岛）的西岱宫更适合居住，于是搬迁至此。在他之后的法国国王再度搬出罗浮宫，直至1546年，弗朗索瓦一世才成为居住在罗浮宫的第二位国王。弗朗索瓦一世除了曾从意大利购买了包括油画《蒙娜丽莎》在内的大量艺术品外，还将原始的中世纪建筑夷为平地，命令建筑师皮埃尔·勒柯按照文艺复兴风格对其加以改建，于1546年至1559年修建了今日罗浮宫建筑群最东端的卡利庭院。扩建工程一直持续到亨利二世登基。亨利二世去世后，王太后凯瑟琳·德·美第奇集中力量修建杜伊勒里宫及杜乐丽花园，对罗浮宫的扩建工作再度停止。\n\n\n目前仅存的中世纪地下室内的原城堡护城河遗迹。', metadata={'source': './demo2.txt'}),
  Document(page_content='21世纪\n1995年，法国总统贾克·席哈克在其朋友、艺术品收藏家和经销商雅克·克恰什(Jacques